# Composizione Multipla

Il punto d'arrivo dell'analisi dati e di frequente quello di divulgare i risultati della propria analisi tramite dei grafici, che permettano di riassumere, in poco tempo, quanto riscontrato. 

Ecco che quindi, la capacita di saper costruire una *dashboard* diventa un tassello fondamentale per un data scientist. 

<center>
<img src="images/dashboard.avif" width="300">
</center>

In questo notebook vedremo come sia possibile combinare piu grafici, ottenendo una composizione multipla in *Altair*. Di preciso vedremo: 
1. *layer*: sovrapposizione di grafici,
2. *facet*: partizionamento dei dati in grafici multipli, 
3. *concatenate*: posizionamento dei grafici in uno spazio condiviso,
4. *repeat*: ripetizione con parametri diversi dello stesso grafico. 

<center>
<img src = "images/AlgebraCompositiva.png" width="1000">
</center>

Per prima cosa, importiamo le librerie fondamentali per il funzionamento di *Altair*:

In [6]:
import pandas as pd
import altair as alt

Utilizzeremo dati atmosferici raccolti per le citta di New York e Seattle. 

In [7]:
weather = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/weather.csv'
df = pd.read_csv(weather)
df.head(10)

,location,date,precipitation,temp_max,temp_min,wind,weather
0,Seattle,2012-01-01,0.0,12.8,5.0,4.7,drizzle
1,Seattle,2012-01-02,10.9,10.6,2.8,4.5,rain
2,Seattle,2012-01-03,0.8,11.7,7.2,2.3,rain
3,Seattle,2012-01-04,20.3,12.2,5.6,4.7,rain
4,Seattle,2012-01-05,1.3,8.9,2.8,6.1,rain
5,Seattle,2012-01-06,2.5,4.4,2.2,2.2,rain
6,Seattle,2012-01-07,0.0,7.2,2.8,2.3,rain
7,Seattle,2012-01-08,0.0,10.0,2.8,2.0,sun
8,Seattle,2012-01-09,4.3,9.4,5.0,3.4,rain
9,Seattle,2012-01-10,1.0,6.1,0.6,3.4,rain


## 🧅 *Layer*

Questa prima operazione permette di *sovrapporre* due o piu grafici. Nello specifico questa operazione permette di sovrappore *mark* diversi nello stesso grafico. 

Affinche quest'operazione sia possibile e necessario che gli assi che i grafici sovrapposti condivideranno siano *compatibili*. Qualora questo non acadesse possiamo ovviare al problema creando grafici a doppio asse. 

<center>
<img src = "images/onion.jpg" width="350">
</center>

### Asse Condiviso

Per prima cosa tracciamo un grafico sovrapposto ad asse condiviso. In questo caso i grafici sovrapposti hanno assi compatibili, ovvero mappati su **campi compatibili** del dataset.  

In [8]:
alt.Chart(weather).mark_area().encode(
  alt.X('month(date):T'),
  alt.Y('average(temp_max):Q'),
  alt.Y2('average(temp_min):Q')
)

alt.Chart(...)

> 💀 **__Attenzione__** 
>
> Il grafico di sopra considera la media della temperatura minima e massima di **entrambe** le citta, perdende un'informazione fondamentale nei nostri dati. 

In [9]:
alt.Chart(weather).mark_area(opacity=0.3).encode(
  alt.X('month(date):T'),
  alt.Y('average(temp_max):Q'),
  alt.Y2('average(temp_min):Q'),
  alt.Color('location:N')
)

alt.Chart(...)

Possiamo anche visualizzare un grafico che mostri la temperatura media annuale per entrambe le citta. Poi possiamo fondere i grafici cosi ottenuti, ottenendone uno piu completo. 

In [ ]:
alt.Chart(weather).mark_line().transform_calculate(
  temp_mid='(+datum.temp_min + +datum.temp_max) / 2' # temperatura media
).encode(
  alt.X('month(date):T'),
  alt.Y('average(temp_mid):Q'),
  alt.Color('location:N')
)

alt.Chart(...)

> 👀 **__Sintassi__**
>
> La sintassi `+datum.temp_min` specifica, col `+` incipitale, che *Altair* deve trrattare il valore *temp_min* nel dataset come una valore **numerico**. Questo problema non si sarebbe posto qualora avessimo utilizzato un *pandas* `DataFrame`. 

In [11]:
tempMinMax = alt.Chart(weather).mark_area(opacity=0.3).encode(
  alt.X('month(date):T'),
  alt.Y('average(temp_max):Q'),
  alt.Y2('average(temp_min):Q'),
  alt.Color('location:N')
)

tempMid = alt.Chart(weather).mark_line().transform_calculate(
  temp_mid='(+datum.temp_min + +datum.temp_max) / 2'
).encode(
  alt.X('month(date):T'),
  alt.Y('average(temp_mid):Q'),
  alt.Color('location:N')
)

tempMinMax + tempMid

alt.LayerChart(...)

Infine non rimane che ripulire il grafico che abbiamo ottenuto. Possiamo: 
1. dare un nome all'asse y piu conciso
2. rimuovere il titolo dell'asse x

In [12]:
tempMinMax = alt.Chart(weather).mark_area(opacity=0.3).encode(
  alt.X('month(date):T', title=None, axis=alt.Axis(format='%b')),  
  alt.Y('average(temp_max):Q', title='Avg. Temperature °C'), # basta specificare il titolo per uno dei Chart sovrapposti
  alt.Y2('average(temp_min):Q'),
  alt.Color('location:N')
)

tempMid = alt.Chart(weather).mark_line().transform_calculate(
  temp_mid='(+datum.temp_min + +datum.temp_max) / 2'
).encode(
  alt.X('month(date):T'),
  alt.Y('average(temp_mid):Q'),
  alt.Color('location:N')
)

tempMinMax + tempMid
# equivalente a: alt.layer(tempMinMax, tempMid)

alt.LayerChart(...)

### Doppio Asse 

Qualora si volessere sovrappore due grafici con assi non compatibili, possiamo risolvere il problema facendo in modo che nel grafico risultanto ci sia un doppio asse. 

Proviamo a rispondere a questa domanda: *possiamo definere Seattle una citta piovosa?*

In [13]:
alt.Chart(weather).transform_filter(
  'datum.location == "Seattle"' # trasformata di filtraggio 
).mark_line(
  interpolate='monotone',
  stroke='grey'
).encode(
  alt.X('month(date):T', title=None),
  alt.Y('average(precipitation):Q', title='Precipitation')
)

alt.Chart(...)

Adesso vorremo paragonare i valori delle precipitazioni con i valori di temperatura nel dataset. Per fare questo potremmo provare a sovrappore i grafici come prima. Tuttavia il grafico risultato mostra il problema della differenza di scala fra precipitazioni e temperatura. Infatti, nonostante siano entrambi valori numerici, la loro differenza di scala rende il grafico risultato poco utile. 

In [14]:
tempMinMax = alt.Chart(weather).transform_filter(
  'datum.location == "Seattle"'
).mark_area(opacity=0.3).encode(
  alt.X('month(date):T', title=None, axis=alt.Axis(format='%b')),
  alt.Y('average(temp_max):Q', title='Avg. Temperature °C'),
  alt.Y2('average(temp_min):Q')
)

precip = alt.Chart(weather).transform_filter(
  'datum.location == "Seattle"'
).mark_line(
  interpolate='monotone',
  stroke='grey'
).encode(
  alt.X('month(date):T'),
  alt.Y('average(precipitation):Q', title='Precipitation')
)

alt.layer(tempMinMax, precip)

alt.LayerChart(...)

Temperatura e precipitazioni hanno **unita di misura diverse** quindi, nonostante siano entrambi valori numerici, comunque non possono condividere lo stesso asse. 

Per ottenere un grafico con doppio asse dobbiamo specificare come *Altair* deve risolvere il problema degli assi condivisi fra i grafici sovrapposti. 

In [15]:
tempMinMax = alt.Chart(weather).transform_filter(
  'datum.location == "Seattle"'
).mark_area(opacity=0.3).encode(
  alt.X('month(date):T', title=None, axis=alt.Axis(format='%b')),
  alt.Y('average(temp_max):Q', title='Avg. Temperature °C'),
  alt.Y2('average(temp_min):Q')
)

precip = alt.Chart(weather).transform_filter(
  'datum.location == "Seattle"'
).mark_line(
  interpolate='monotone',
  stroke='grey'
).encode(
  alt.X('month(date):T'),
  alt.Y('average(precipitation):Q', title='Precipitation')
)

alt.layer(tempMinMax, precip).resolve_scale(y='independent') # metodo di risoluzione

alt.LayerChart(...)

Esiste un metodo per rendere il codice che abbiamo scritto piu chiaro e mantenibile. Possiamo definere la *forma* del grafico senza specificare quale dataset debba essere utilizzato pe la sua creazione. 

Successivamente, con il parametro `data` della funzione `layer`, possiamo definire quale sia il dataset a partire dal quale, tramite le funzioni di *scale* definite nei grafici singoli, costruire il grafico finale. 

In [16]:
tempMinMax = alt.Chart().mark_area(opacity=0.3).encode(
  alt.X('month(date):T', title=None, axis=alt.Axis(format='%b')),
  alt.Y('average(temp_max):Q', title='Avg. Temperature °C'),
  alt.Y2('average(temp_min):Q')
)

precip = alt.Chart().mark_line(
  interpolate='monotone',
  stroke='grey'
).encode(
  alt.X('month(date):T'),
  alt.Y('average(precipitation):Q', title='Precipitation')
)

alt.layer(tempMinMax, precip, data=weather).transform_filter(
  'datum.location == "Seattle"'
).resolve_scale(y='independent')

alt.LayerChart(...)

Un problema rilevante dei grafici a doppio asse e la loro **interpratibilita**. Infatti sono facilmente proni a cattivi interpretazione ed affollamento grafico. Qualora possibile, si consiglia di evitarne l'utilizzo. 

## 💎 *Facet*

Il *faceting* consiste nella suddivisione del dataset in sottogruppi e nella visualizzazione separata di ognuno di questi. 

<center>
<img src = "images/diamond.jpg" width="350">
</center>

Nonostante esista un operatore `facet` specifico in *Altair*, gli *encoding channel* `row` e `column` possono creare un sottinsieme dei grafici creabili con l'operatore originale. 

In [17]:
# istogramma delle temperature massime a Seattle 
alt.Chart(weather).mark_bar().transform_filter(
  'datum.location == "Seattle"'
).encode(
  alt.X('temp_max:Q', bin=True, title='Temperature (°C)'),
  alt.Y('count():Q')
)

alt.Chart(...)

*Come cambia questo profilo di temperatura in funzione del tempo atmosferico del giorno considerato?*

In [18]:
colors = alt.Scale( # colori da assegnare ad ognuno degli istogrammi creati
  domain=['drizzle', 'fog', 'rain', 'snow', 'sun'],
  range=['#aec7e8', '#c7c7c7', '#1f77b4', '#9467bd', '#e7ba52']
)

alt.Chart(weather).mark_bar().transform_filter(
  'datum.location == "Seattle"'
).encode(
  alt.X('temp_max:Q', bin=True, title='Temperature (°C)'),
  alt.Y('count():Q'),
  alt.Color('weather:N', scale=colors),
  alt.Column('weather:N') # colunm encoding channel
).properties(
  width=100,
  height=100
)

alt.Chart(...)

Ricreiamo il grafico sopra, ma questa volta utilizzando il metodo `facet`. 

Iniziamo con la stessa definizione di istogramma di base, ma rimuoviamo la sorgente dati, la trasformazione del filtro e il canale colonna. 

Possiamo quindi invocare il metodo `facet`, passando i dati e specificando che dobbiamo suddividerli in colonne in base al campo meteorologico. Il metodo `facet` accetta come argomenti sia `row` che `column`. I due possono essere utilizzati insieme per creare una griglia 2D di grafici sfaccettati.

In [19]:
colors = alt.Scale(
  domain=['drizzle', 'fog', 'rain', 'snow', 'sun'],
  range=['#aec7e8', '#c7c7c7', '#1f77b4', '#9467bd', '#e7ba52']
)

alt.Chart().mark_bar().encode(
  alt.X('temp_max:Q', bin=True, title='Temperature (°C)'),
  alt.Y('count():Q'),
  alt.Color('weather:N', scale=colors)
).properties(
  width=100,
  height=100
).facet(
  data=weather,
  column='weather:N'
).transform_filter(
  'datum.location == "Seattle"'
)

alt.FacetChart(...)

Ma se con `facet` ottengo lo stesso risultato che otterrei utilizzando solamete i *channel* `row` e `column`, perche lo dovrei usare? 

In [20]:
tempMinMax = alt.Chart().mark_area(opacity=0.3).encode(
  alt.X('month(date):T', title=None, axis=alt.Axis(format='%b')),
  alt.Y('average(temp_max):Q', title='Avg. Temperature (°C)'),
  alt.Y2('average(temp_min):Q'),
  alt.Color('location:N')
)

tempMid = alt.Chart().mark_line().transform_calculate(
  temp_mid='(+datum.temp_min + +datum.temp_max) / 2'
).encode(
  alt.X('month(date):T'),
  alt.Y('average(temp_mid):Q'),
  alt.Color('location:N')
)

# facet e in certi casi esplicitamente utile
alt.layer(tempMinMax, tempMid).facet(
  data=weather,
  column='location:N'
)

alt.FacetChart(...)

I grafici che abbiamo ottenuto **condividono** lo stesso asse y. Tuttavia potremo voler accorpare in questo modo due grafici con: 
- assi diversi (`resolve_axis`)
- *scale* diverse (`resolve_scale`)

In [21]:
tempMinMax = alt.Chart().mark_area(opacity=0.3).encode(
  alt.X('month(date):T', title=None, axis=alt.Axis(format='%b')),
  alt.Y('average(temp_max):Q', title='Avg. Temperature (°C)'),
  alt.Y2('average(temp_min):Q'),
  alt.Color('location:N')
)

tempMid = alt.Chart().mark_line().transform_calculate(
  temp_mid='(+datum.temp_min + +datum.temp_max) / 2'
).encode(
  alt.X('month(date):T'),
  alt.Y('average(temp_mid):Q'),
  alt.Color('location:N')
)

alt.layer(tempMinMax, tempMid).facet(
  data=weather,
  column='location:N'
).resolve_axis(y='independent') # assi indipendenti 

alt.FacetChart(...)

In [22]:
tempMinMax = alt.Chart().mark_area(opacity=0.3).encode(
  alt.X('month(date):T', title=None, axis=alt.Axis(format='%b')),
  alt.Y('average(temp_max):Q', title='Avg. Temperature (°C)'),
  alt.Y2('average(temp_min):Q'),
  alt.Color('location:N')
)

tempMid = alt.Chart().mark_line().transform_calculate(
  temp_mid='(+datum.temp_min + +datum.temp_max) / 2'
).encode(
  alt.X('month(date):T'),
  alt.Y('average(temp_mid):Q'),
  alt.Color('location:N')
)

alt.layer(tempMinMax, tempMid).facet(
  data=weather,
  column='location:N'
).resolve_scale(y='independent') # scale indipendenti

alt.FacetChart(...)

> 💀 **__Attenzione__** 
>
> Il grafico sopra e **fuorviante**. Infatti le *scale* dell'asse y sono diverse, nonostante di primo impatto i due grafici paiano direttamente comparabili!

## 🔗 Concatenate

Questa operazione permette di concatenare grafici completamente **diversi** fra loro all'interno della stessa immagine, a differenza dell'operatore `facet` che permetteva di crare visualizzazione specifiche per sottogruppi dello stesso dataset.

<center>
<img src = "images/mondrian.jpg" width="350">
</center>

Esistono degli operatori appositi che permettono di svolegere queste operazioni velocemente: 
- concatenazione orizzontale (`hconcat`): `|`
- concatenazione verticale (`vconcac`): `&`

Per fare un esempio possiamo definere un `Chart` con tutte le caratteristiche di base che vogliamo all'interno degli altri grafici e poi declinarlo a seconda del risultato che vogliamo ottenere con l'operatore `encode`. 

In [28]:
base = alt.Chart(weather).mark_line().encode(
  alt.X('month(date):T', title=None),
  color='location:N'
).properties(
  width=240,
  height=180
)

temp = base.encode(alt.Y('average(temp_max):Q'))
precip = base.encode(alt.Y('average(precipitation):Q'))
wind = base.encode(alt.Y('average(wind):Q'))

(temp | precip) & wind

alt.VConcatChart(...)

## 🔁 Repeat

La potenza dell'operazione *concatenate* e quella di combinare grafici di diversa provenienza all'interno della stessa visualizzazione d'insieme. 

<center>
<img src = "images/ants.webp" width="350">
</center>

Questa operazione permette di utilizzare un *Chart* come **template**. L'idea e quella di definire uno scheletro di base che poi viene declinato modificado parametri specifici ad ogni ripetizione tramite l'operatore `repeat`. I grafici risultato vengono poi visualizzati:
- in **colonna**, se si e deciso di utilizzare il parametro `row`,
- in **riga**, con il parametro `column`.

In [26]:
alt.Chart(weather).mark_line().encode(
  alt.X('month(date):T',title=None),
  alt.Y(alt.repeat('row'), aggregate='average', type='quantitative'), # operatore repeat
  color='location:N'
).properties(
  width=200,
  height=150
).repeat(
  row=['temp_max', 'precipitation', 'wind'] # specificata lista di colonne da considerare
)

alt.RepeatChart(...)

Possiamo anche utilizzare i parametri `row` e `column` insieme. In questo caso verrano considerate tutte le permutazioni possibili dei valori presenti nelle liste `row` and `column` per tracciare i grafici che poi saranno visualizzati a schermo. 

In [27]:
# scatter plot matrix (SPLOM) per la citta di Seattle
alt.Chart().mark_point(filled=True, size=10, opacity=0.1, color = '#8c78f9').encode(
  alt.X(alt.repeat('column'), type='quantitative'), # i
  alt.Y(alt.repeat('row'), type='quantitative') # j
).properties(
  width=100,
  height=100
).repeat(
  data=weather,
  row=['wind', 'precipitation', 'temp_max', 'temp_min'], # valori j
  column=['wind', 'precipitation', 'temp_max', 'temp_min'] # valori i 
).transform_filter(
  'datum.location == "Seattle"'
)

alt.RepeatChart(...)

> **Interpretazione**
>
> Osservando questi grafici, non sembra esserci una forte associazione tra precipitazioni e vento, sebbene si osservi che eventi estremi di vento e precipitazioni si verificano in intervalli di temperatura simili (~5-15 °C). Tuttavia, questa osservazione non è particolarmente sorprendente: se rivisitiamo l'istogramma all'inizio della sezione delle faccette, possiamo chiaramente vedere che i giorni con temperature massime comprese tra 5 e 15 °C sono i più comuni.

## 🧮 Un'algebra per la composizione di grafici

Utilizzando insieme tutti gli operatori che abbiamo appena visto e possibile creare una *dashboard* abbastanza complessa. 

<center>
<img src = "images/afar.webp" width="300">
</center>

In [29]:
# matrice di scatter plot
splom = alt.Chart().mark_point(filled=True, size=15, opacity=0.5).encode(
  alt.X(alt.repeat('column'), type='quantitative'),
  alt.Y(alt.repeat('row'), type='quantitative')
).properties(
  width=125,
  height=125
).repeat(
  row=['wind', 'precipitation', 'temp_max'],
  column=['wind', 'precipitation', 'temp_max']
)

# istogramma di tempertura, precipitazioni, vento medi per mese con linea per la media 
dateHist = alt.layer(
  alt.Chart().mark_bar().encode(
    alt.X('month(date):O', title='Month'),
    alt.Y(alt.repeat('row'), aggregate='average', type='quantitative')
  ),
  alt.Chart().mark_rule(stroke='firebrick').encode(
    alt.Y(alt.repeat('row'), aggregate='average', type='quantitative')
  )
).properties(
  width=175,
  height=125
).repeat(
  row=['temp_max', 'precipitation', 'wind']
)

# facet di un istogramma della temperatura, raggruppati e colorati per weather per riga  
tempHist = alt.Chart(weather).mark_bar().encode(
  alt.X('temp_max:Q', bin=True, title='Temperature (°C)'),
  alt.Y('count():Q'),
  alt.Color('weather:N', scale=alt.Scale(
    domain=['drizzle', 'fog', 'rain', 'snow', 'sun'],
    range=['#aec7e8', '#c7c7c7', '#1f77b4', '#9467bd', '#e7ba52']
  ))
).properties(
  width=115,
  height=100
).facet(
  column='weather:N'
)

# concatenazione per la creazione di una dashboard
alt.vconcat(
  alt.hconcat(splom, dateHist),
  tempHist,
  data=weather,
  title='Seattle Weather Dashboard'
).transform_filter(
  'datum.location == "Seattle"'
).resolve_legend(
  color='independent'
).configure_axis(
  labelAngle=0
)

alt.VConcatChart(...)

In [30]:
# matrice di scatter plot
splom = alt.Chart().mark_point(filled=True, size=15, opacity=0.5).encode(
  alt.X(alt.repeat('column'), type='quantitative'),
  alt.Y(alt.repeat('row'), type='quantitative')
).properties(
  width=125,
  height=125
).repeat(
  row=['wind', 'precipitation', 'temp_max'],
  column=['wind', 'precipitation', 'temp_max']
)

# istogramma di tempertura, precipitazioni, vento medi per mese con linea per la media 
dateHist = alt.layer(
  alt.Chart().mark_bar().encode(
    alt.X('month(date):O', title='Month'),
    alt.Y(alt.repeat('row'), aggregate='average', type='quantitative')
  ),
  alt.Chart().mark_rule(stroke='firebrick').encode(
    alt.Y(alt.repeat('row'), aggregate='average', type='quantitative')
  )
).properties(
  width=175,
  height=125
).repeat(
  row=['temp_max', 'precipitation', 'wind']
)

# facet di un istogramma della temperatura, raggruppati e colorati per weather per riga  
tempHist = alt.Chart(weather).mark_bar().encode(
  alt.X('temp_max:Q', bin=True, title='Temperature (°C)'),
  alt.Y('count():Q'),
  alt.Color('weather:N', scale=alt.Scale(
    domain=['drizzle', 'fog', 'rain', 'snow', 'sun'],
    range=['#aec7e8', '#c7c7c7', '#1f77b4', '#9467bd', '#e7ba52']
  ))
).properties(
  width=115,
  height=100
).facet(
  column='weather:N'
)

# concatenazione per la creazione di una dashboard
alt.vconcat(
  alt.hconcat(splom, dateHist),
  tempHist,
  data=weather,
  title='New York Weather Dashboard'
).transform_filter(
  'datum.location == "New York"'
).resolve_legend(
  color='independent'
).configure_axis(
  labelAngle=0
)

alt.VConcatChart(...)